In [2]:
%load_ext autoreload
%autoreload
%matplotlib inline

import numpy as np
import pandas as pd
import gensim
from utils import *

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Loading data

In [3]:
# Load data
market_GOOG = load_prices("C:\\Users\\Vojta-Acer\Desktop\\Diplomka\\dataMarket\\GOOG.csv")
tweets_GOOG = load_tweets("C:\\Users\\Vojta-Acer\\Desktop\\Diplomka\\dataProcessed\\tweetsGOOG.csv")
tweets_GOOG_5min = aggregate_tweets(tweets_GOOG, '5min', 'tokens')
tweets_AAPL = load_tweets("C:\\Users\\Vojta-Acer\\Desktop\\Diplomka\\dataProcessed\\tweetsAAPL.csv")
tweets_AAPL_5min = aggregate_tweets(tweets_AAPL, '5min', 'tokens')

In [4]:
# Load Embedding data
W2V = gensim.models.KeyedVectors.load_word2vec_format("E:\diplomka temp\word2vec\glove.twitter.27B.200d.txt")
emb_5min_GOOG = tweets_GOOG_5min['text'].apply(tweet2vec_mean, args=[W2V])
emb_5min_GOOG = emb_5min_GOOG.apply(pd.Series)
emb_5min_AAPL = tweets_AAPL_5min['text'].apply(tweet2vec_mean, args=[W2V])
emb_5min_AAPL = emb_5min_GOOG.apply(pd.Series)

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\diplomka temp\\word2vec\\glove.twitter.27B.200d.txt'

## Creating labels from market data

First of all, we restrict market data to trading hours and bussiness days, otherwise we would get bias because both tweets and stocks follow the same weed and day/night cycles. 


In [ ]:
#market = market_master.copy()
#market = market_master[market_master['Tick Count'] > 0]
market_GOOG['TC 5min'] = market_GOOG['Tick Count'].rolling(5).mean()
market_GOOG['TC 60min'] = market_GOOG['Tick Count'].rolling(60).mean()
market = market_GOOG.between_time('9:30', '16:00')

In [ ]:
# Plots
market['Tick Count'].plot(use_index=False, title = 'Tick count for each minute')

Plots of tick count shows, that there is still some seasonality present. Significant positive outlyers are last minutes of trading day.


## Goals of this section is to create labels from market data
ideas for labels, we can model many different things, such as:

#### Change in price
Try to model change up/down of prices after/before some time
- failed, just noise is there

#### Up Tick minus Down Tick

###### Construction
First, we subtrack down tick from uptick on minute frequency. This value is then split to three categories, based on its quantiles: 0 - 0.33 - 0.66 - 1. Upside of this approach to labeling is having three - category label with equal class distribution as result. Then, we **add lag/lead to the label** and we are ready for modeling.

###### Modeling
In the example, we use 100 minutes of lag to the label. We use twitter minute data as text data. 

Vectorizer is simple Count vectorizer, which is about twice as better (cca 0.01 Kappa) as log TF-IDF with smoothing, which should be the best vectorizer according to IR theory (it has 0.005 Kappa)
Used model is Multinomial NB, as it is fast and reasonably good with high dim data

Since we have equally distributed labels to classes, accuracy is good measure as well

- Results: only cca 1% kappa, so it can be noise
- Results: Results are NOT CHANGING with adding lag or lead to the label (!!!)


In [ ]:
label = (market_GOOG['Up Ticks'] - market_GOOG['Down Ticks']).rename('label')
label.index = label.index + pd.DateOffset(minutes = 10)
features = tweets_GOOG_5min.join(label, how = 'inner').dropna() # we use Twitter 5 minute data


In [ ]:

features['label'] = pd.qcut(features['label'], [0, 0.25, 0.66, 1.])  # three category label
features = features.sample(frac=1) # shuffle


# Modeling
vec = CountVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, binary=False)
label = features['label'].astype(str)
train = {}
train['Modeling using text'] = vec.fit_transform(features['text'])
train['Modeling using twitter count'] = pd.DataFrame(features['tweet_count'])
train['Modeling using date/hour/minute'] = pd.DataFrame([features.index.hour.astype(str),
                                                   features.index.minute.astype(str), 
                                                   features.index.dayofweek.astype(str)]).T
for key in train:
    pred = cross_val_predict(MultinomialNB(), train[key], label, cv=5, n_jobs=1, verbose=0)
    print(key)
    evaluate(pred, label)
    print('\n')

#### Tick Count

###### Construction
Similar to previous, but we use Total Tick count in given minute to construct the categories. Then similar as previous.

Value is split to four categories, based on quantiles: 0 - 0.25 - 0.5 - 0.75 - 1. Upside of this approach to labeling is having four - category label with equal class distribution as result. Then, we **add lag/lead to the label** and we are ready for modeling.


###### Modeling
Again, we use 100 minutes of lag to the label with twitter minute data as text data. Vectorizer is log TF IDF with smoothing (here is slightly better).

Used model is Multinomial NB, as it is fast and reasonably good with high dim data

- Results: Reasonable accuracy and Kappa, even in the 4 category setting (3 category is even better)
- Experiments: lower twitter frequency means lower kappa/accuracy
- Vectorizers are sensitive to terrible settings (for example that filtering 90% observations means bad results)
- Again, with adding lag or lead to the label the results are NOT CHANGING as expected: With greater temporal distance from tweet, the label is predicted with better accuracy (!!!)


In [ ]:
label = market['Tick Count'].rename('label')
label.index = label.index + pd.DateOffset(minutes = 600)

features = tweets_5min.join(label, how = 'left').dropna() # we use Twitter minute data
features['label'] = pd.qcut(features['label'], [0, .25, .5, .75, 1.])  # four category label
features = features.sample(frac=1) # shuffle
#features = features.sample(frac=1).reset_index(drop=True)

# Modeling
vec = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, sublinear_tf = True, use_idf=True)
label = features['label'].astype(str)
train = {}
train['Modeling using text'] = vec.fit_transform(features['text'])
train['Modeling using twitter count'] = pd.DataFrame(features['tweet_count'])
train['Modeling using date/hour/minute'] = pd.DataFrame([features.index.hour.astype(str),
                                                   features.index.minute.astype(str), 
                                                   features.index.dayofweek.astype(str)]).T
for key in train:
    pred = cross_val_predict(MultinomialNB(), train[key], label, cv=5, n_jobs=1, verbose=0)
    print(key)
    evaluate(pred, label)
    print('\n')

# Regression

In [ ]:
# Adjustment
df = pd.DataFrame(market['TC 5min'].rename('label'))
df['day'] = df.index.dayofweek
df['day'] = df.groupby(by = 'day')['label'].transform(np.median)

df['hour'] = df.index.hour
df['hour'] = df.groupby(by = 'hour')['label'].transform(np.median)

df['label_adj'] = df['label'] - df['hour']
target = df['label_adj']
target.index = target.index + pd.DateOffset(days = 1)

In [ ]:
target = market['TC 5min'].rename('label')
target.index = target.index + pd.DateOffset(days = 1)

In [ ]:
train, label = tweets_AAPL_5min.align(target, axis=0, join='inner')
train, label = shuffle(train, label)
#train = shuffle(train)

vecorizer = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x)
train = vecorizer.fit_transform(train['text'].values)
model = Ridge(alpha = 2)

pred = cross_val_predict(model, train, label, cv=3)
mean_squared_error(label, pred)

In [ ]:
y = pd.DataFrame(label)
y['pred'] = pred
y = y.sort_index()
y.index = y.index.astype(str)
y.rolling(50).mean().plot()

In [ ]:
# Adjustment
df = pd.DataFrame(market['TC 5min'].rename('label'))
df['day'] = df.index.dayofweek
df['day'] = df.groupby(by = 'day')['label'].transform(np.median)

df['hour'] = df.index.hour
df['hour'] = df.groupby(by = 'hour')['label'].transform(np.median)

df['label_adj'] = df['label'] - df['hour']
target = df['label_adj']
target.index = target.index + pd.DateOffset(hours = 240)

train, label = tweets_5min.align(target, axis=0, join='inner')
vecorizer = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x)
train = vecorizer.fit_transform(train['text'].values)

d = np.ones(0)
model = Ridge(alpha = 5)
window = 10
for i in range(1000, 12000, window):
    model.fit(train[:i], label[:i])
    pred = model.predict(train[i:i+window])
    d = np.append(d, pred)
    print(i)

In [ ]:
model.fit(train, label)

In [ ]:
coef = pd.Series(model.coef_)
inverse = dict((v, k) for k, v in vecorizer.vocabulary_.items())

x = list(coef.nlargest(80).index)
#x = list(coef.nsmallest(20).index)
for item in x:
    print(inverse[item])

In [ ]:
y = pd.DataFrame(label[1000:12000])
y['predictions'] = d
y.index = y.index.astype(str)
y.rolling(10).mean().plot()

In [ ]:
y = pd.DataFrame(label[1000:12010])
y['predictions'] = d
y.index = y.index.astype(str)
y.rolling(500).mean().plot()

In [ ]:
y = pd.DataFrame(label[1000:12000])
y['predictions'] = d
y.index = y.index.astype(str)
y.rolling(200).mean().plot()

In [ ]:
model.fit(train[:1000], label[:1000])
pred = model.predict(train[1000:1000+i])
df = pd.DataFrame(label[1000:1000+i])
df['prediction'] = pred

df.sort_index().rolling(100).mean().plot()

# Embbedings

In [ ]:
target = market['TC 5min'].rename('label')
target.index = target.index + pd.DateOffset(hours = 5)

train, label = emb_5min_A.align(target, axis=0, join='inner')
#train = shuffle(train)
train, label = shuffle(train, label)

model = Ridge(alpha = 0.1)
pred = cross_val_predict(model, train, label, cv=5)
mean_squared_error(label, pred)

In [ ]:
y = pd.DataFrame(label)
y['pred'] = pred
y = y.sort_index()
y.index = y.index.astype(str)
y.rolling(100).mean().plot()

In [ ]:
mean_squared_error(label, x['mean'])

In [ ]:
x = pd.DataFrame(label.index)
x['mean'] = label.mean()

In [ ]:
g = {}
for i in list(range(-300, 300, 1)):

    target = market['TC 5min'].rename('label')
    target.index = target.index + pd.DateOffset(days = i)

    train, label = emb_5min_GOOG.align(target, axis=0, join='inner')
    train, label = shuffle(train, label)

    model = Ridge(alpha = 5)
    pred = cross_val_predict(model, train, label, cv=5)
    g[i] = mean_squared_error(label, pred)

In [ ]:
y = pd.DataFrame(g, index = [0]).T
y = y.reindex(y.index[::-1])
y.index = -1*(y.index)

plt.plot(y[0], color = 'black')


In [ ]:
y = pd.DataFrame(label)
y['pred'] = pred
y = y.sort_index()
y.index = y.index.astype(str)
y.rolling(300).mean().plot()

# Modeling Variance


In [ ]:
label = market['Close'].rolling(5).std().rename('label')
#label = market['Close'].rename('label')

label.index = label.index + pd.DateOffset(minutes = 10)
label = label.replace([np.inf, -np.inf], np.nan)
features = tweets_5min.join(label, how = 'left').dropna()
#features = features.sample(frac=1, replace = False) # shuffle

# Modeling
vec = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, sublinear_tf = True, use_idf=True)
#vec = CountVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, binary=False)


label = features['label']
train = vec.fit_transform(features['text'])

#model = LinearRegression()
model = Ridge(alpha = 1)

pred = cross_val_predict(model, train, label, cv=5, n_jobs=1, verbose=0)
mean_squared_error(label, pred)

In [ ]:
y = pd.DataFrame(label)

y['pred'] = pred
y = y.sort_index()
y.index = y.index.astype(str)
y.rolling(100).mean().plot()

# Variance daily

In [ ]:
dataset.rolling(1).mean().plot()

In [ ]:
dataset['tweet_count'] = np.log(dataset['tweet_count'])
